## 05-3 트리의 앙상블

### 랜덤 포레스트

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 와인 데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나눔.
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# RandomForestClassifier()
# : 기본적으로 전체 특성 개수의 제곱근만큼의 특성 선택 / 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용.
# : 기본적으로 100개의 결정 트리를 사용하므로 n_jobs = -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋음.
# : 자체적으로 모델을 평가하는 점수 얻을 수 있음.
# RandomForestRegressor() : 전체 특성 사용.
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# cross_validate() : 교차 검증 수행
# n_jobs = -1로 지정하여 최대한 병렬로 교차 검증 수행.
#  return_train_score=True로 지정(기본값 : False)하여 검증 점수 뿐 아니라 훈련 세트에 대한 점수까지 반환. 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이.
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 훈련 세트에 다소 과대적합.


# 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassfier가 제공하는 중요한 매개변수를 모두 제공.(결정 트리의 큰 장점 중 하나인 특성 중요도 계산)
# 랜덤 포레스트의 특성 중요도 == 각 결정 트리의 특성 중요도를 취합한 것.

# 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음.
# 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움을 줌.

# oob_score=True : 각 결정 트리의 OOB 점수를 평균하여 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
# 교차 검증에서 얻은 점수와 비슷한 결과.

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406


## 엑스트라 트리

In [3]:
# ExtraTreesClassifier() : 사이킷런에서 제공하는 엑스트라 트리
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 랜덤 포레스트와 비슷한 결과.

et.fit(train_input, train_target)
print(et.feature_importances_)
# 특성 중요도 제공
#[알코올 도수, 당도, ph]
# 엑스트라 트리도 결정 트리보다 당도에 대한 의존성이 작음.

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 거의 과대적합 되지 않음.

# n_estimators=500 : 결정 트리 개수를 500개로 설정
# learning_rate 기본값 = 0.1
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 결정 트리 개수를 5배나 늘렸지만 과대적합 잘 억제하고 있음.

gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 특성 중요도
# 랜덤 포레스트 보다 당도에 더 집중.

0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅

In [5]:
from sklearn.experimental import enable_hist_gradient_boosting
# HistGradientBoostingClassifier 
# : 사이킷런의 히스토그램 기반 그레이디언트 부스팅 클래스, 기본 매개변수에서 안정적인 성능을 얻을 수 있음.
# : 트리 개수 지정시 부스팅 반복 횟수 지정하는 max_iter 사용.
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
# permutation_importance()
# : 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산
# : 특성을 하나씩 랜덤하게 섞어 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
# : 훈련 세트 뿐 아니라 테스트 세트에도 적용가능, 사이킷런에서 제공하는 추정기 모델에 모두 사용가능.
# : 리턴하는 객체는 반복하여 얻은 특성 중요도(importance), 평균 (importance_mean), 표준편차(importances_std)를 담고 있음.
# n_repeats : 랜덤하게 섞을 횟수 지정.

# 훈련 세트에서 특성 중요도 계산
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
# 평균 출력 : 랜덤 포레스트와 비슷한 비율

# 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
# 그레디언트 부스팅과 비슷하게 조금 더 당도에 집중

# 테스트 세트에서의 성능 최종 확인
hgb.score(test_input, test_target)
# 약 87%의 정확도
# 앙상블 모델이 단일 결정 모델보다 좋은 결과 얻음.

0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923

## XGBoost

### 사이킷런 이외에 히스토그램 기반 그레이디언트 부스팅 알고리즘 구현한 라이브러리 1

In [6]:
# XGBoost 사용해 와인 데이터의 교차 검증 점수 확인.
rom xgboost import XGBClassifier

# tree_method='hist' : 히스토그램 기반 그레이디언트 부스팅 사용가능.
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


## LightGBM

### 사이킷런 이외에 히스토그램 기반 그레이디언트 부스팅 알고리즘 구현한 라이브러리 2

In [7]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
